In [100]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../inc')


In [ ]:
# Load raw lodging list
df = pd.read_csv('../DABA/서울시 관광숙박업 인허가 정보.csv', encoding='cp949')
df.info()

In [ ]:
# select item
sel_colum = [2,3,4,8,9,10,11,15,16,17,18,23,24]
sel_df = df.iloc[:,sel_colum]
sel_df.info()


In [116]:
# .. 카카오API로 위경도 삽입, pyproj로 시도하였으나 값이 약간 다름

import requests # 온라인으로 보내거나 받아올 때 사용, 파이썬으로 웹사이트 연동할 때
from urllib.parse import quote # url에서 한글 코드를 변환


# API 키 가져오기
filename = '/Users/mankikim/Desktop/프로젝트/더조은/API/kakao_API.txt'
with open(filename) as f:
    api_key = f.read().strip()
    
len(api_key)  # 32글자가 되야됨, 파일키 정상 문자열개수
     
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lon_list, err_list = [], [], []
for i in sel_df.index:
#for i in [37]:
    temp = quote(sel_df.지번주소[i])
    url = f'{search_url}?query={temp}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
    # print(result)
    # print(i, float(result['documents'][0]['x']), float(result['documents'][0]['y']))
    try:
        lon_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        lon_list.append(np.NAN)
        lat_list.append(np.NAN)
        err_list.append((i,sel_df.지번주소[i]))
    
sel_df['위도'] = lat_list
sel_df['경도'] = lon_list
sel_df.head()

/var/folders/wj/6fqsj4yn655_2rsnl7jt1yyh0000gn/T/ipykernel_13564/958619598.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['위도'] = lat_list
/var/folders/wj/6fqsj4yn655_2rsnl7jt1yyh0000gn/T/ipykernel_13564/958619598.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['경도'] = lon_list


,인허가일자,인허가취소일자,영업상태코드,폐업일자,휴업시작일자,휴업종료일자,재개업일자,지번주소,도로명주소,도로명우편번호,사업장명,좌표정보(X),좌표정보(Y),위도,경도
0,2015-02-03,NaN,1,NaN,NaN,NaN,NaN,서울특별시 강남구 개포동 1229-14,서울특별시 강남구 논현로 66 (개포동),6307.0,호텔 데님(Hotel Denim),204053.596301,441557.179656,37.476190,127.046616
1,2016-05-17,NaN,3,2023-02-28,NaN,NaN,NaN,서울특별시 중구 회현동1가 194-15,"서울특별시 중구 퇴계로 52 (회현동1가, 인송빌딩)",4634.0,티마크그랜드호텔명동,198043.635511,450686.849957,37.558454,126.978646
2,2021-04-06,NaN,1,NaN,NaN,NaN,NaN,서울특별시 금천구 독산동 335-21,서울특별시 금천구 가산로3길 103 (독산동),8526.0,위너스,190183.499829,441092.525925,37.471959,126.889817
3,2014-07-22,NaN,1,NaN,NaN,NaN,NaN,서울특별시 중구 충무로1가 24-30,서울특별시 중구 퇴계로 107 (충무로1가),4536.0,데이즈 호텔 명동,198484.096466,450963.971753,37.560953,126.983631
4,2016-03-29,NaN,2,NaN,2023-01-01,2023-12-31,NaN,서울특별시 강남구 논현동 205-8,"서울특별시 강남구 봉은사로 143 (논현동, 운현오피스텔)",6122.0,강남패밀리호텔,202596.666692,444917.728061,37.506474,127.030156


In [117]:
print(err_list)
print(len(err_list))


[(37, '서울특별시 강남구 논현동 6 6-1'), (44, '서울특별시 강서구 방화동 886 외 6필지'), (73, '서울특별시 관악구 남현동 1061-2 3~7층'), (89, '서울특별시 중구 순화동 214-0 외 1필지'), (94, '서울특별시 성동구 도선동 58-0번지 '), (95, '서울특별시 강북구 수유동 174-32 외1필지(174-19)'), (117, '서울특별시 마포구 합정동 381-17번지 '), (150, '서울특별시 종로구 수송동 51-8 '), (175, '서울특별시 종로구 당주동 29 '), (194, '서울특별시 중구 을지로6가 17-2번지 케레스타 빌딩 18층~20층 '), (212, '서울특별시 중구 북창동 93-99 '), (227, '서울특별시 중구 회현동1가 194-19 '), (233, '서울특별시 중구 광희동2가 360번지 3-7층 '), (238, '서울특별시 중구 광희동2가 43-1 3~6층 '), (266, '서울특별시 중구 태평로1가 51-1 '), (274, '서울특별시 중구 순화동 151번지 외 1필지 C동 '), (280, '서울특별시 용산구 이태원동 119-24번지 119-24,25,108-9,123-10~15,116-1,116-5,112-7,112-15 '), (281, '서울특별시 용산구 이태원동 34-69 36-69, 34-159, 44-64'), (294, '서울특별시 성동구 도선동 72번지 외 2필지(73, 74번지) '), (299, '서울특별시 광진구 광장동 산 21-0번지 '), (315, '서울특별시 광진구 광장동 산 21-0'), (332, '서울특별시 성북구 동선동1가 92-20 외 3필지(92-21, 92-26, 92-27)'), (356, '서울특별시 서대문구 창천동 31-92 8,9,10층'), (401, '서울특별시 강서구 화곡동 918-23번지 외 2필지 '), (447, '서울특별시 영등포구 당산동3가 136번지 외 1필지 '), (574, '서울특별시 강남구 논현동

In [118]:
sel_df.to_csv("../DAIN/crw_list.txt")

In [119]:
err_df = pd.DataFrame(err_list, columns=['index','위경도 missing'])
err_df.to_csv("../DAIN/err_list.txt")